# Training environment information

In [1]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        933M        4.9G        1.2M        6.8G         11G
Swap:            0B          0B          0B


In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-ed05d9d4-69a1-fcbe-7849-b24f7809b0ad)


In [3]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz


In [4]:
!lscpu | grep 'Core(s) per socket:'

Core(s) per socket:  1


In [5]:
!nvidia-smi

Sun Apr  3 06:37:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


Preventing disconnect from Google Colab notebook:
Right-click on the connect button and paste the code to the console of the UI:


```
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click() 
 }
 setInterval(ClickConnect,60000)
```

# Setup

In [7]:
!pip install fastai==2.5.3 -q
!pip freeze

absl-py==1.0.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.28
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill=

In [8]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision.all import *
import sys

In [9]:
# Load EB_ML python libraries
# The following libraries are used in this notebook and should be installed in your local machine before running this notebook.
# eb_colab_utils.py
# eb_ml_battery_lib.py
# eb_ml_utils.py

# path to load external *.py files used in this notebook
# Note: in Google Colab virtual machine you shoud copy the files in "/content" folder after BEFORE running this notebook's cell
external_python_file_path="'/.'"
sys.path.append(external_python_file_path)


from eb_ml_colab_utils import get_root_path,copy_model_to_google_drive
from eb_ml_battery_lib import load_soc_dataset,generate_image_files_from_eis
from eb_ml_utils import save_model_weights,build_data_loader,build_and_train_learner,score_model

# Training Configuration

In [10]:
#configuration dictionary
config ={}

# Root working folder (local or Google Drive)
# config['ROOT_DIR'] = get_root_path("batterie")
config['ROOT_DIR'] = get_root_path("batterie")  

# Folder with dataset in CSV format
#config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets"
config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets/EIS-vs-SOC-2022"

# List of SoC level into dataset
#config['soc_list']=['100','090','080','070','060','050','040','030','020','010']
config['soc_list']=['100','090','080','070','060','050','040','030','020','010']


# Folder to store trained model
#config['MODELS_DIR'] = config['ROOT_DIR']+"/models"
config['MODELS_DIR'] = config['ROOT_DIR']+"/models"

Running on COLAB
Mounted at /gdrive


Source data: battery EIS performed for SoC 100,90,80,70,60,50,40,30,20,10. Same frequnecies for each SoC

Data format:  Re{Zbat}-Im{Zbat}j

```Matlab
prefix = {...
    './Batt1/Meas1/'; --> 1
    './Batt1/Meas2/'; --> 2*
	  './Batt2/Meas1/'; --> 3  
    './Batt2/Meas2/'; --> 4*
    './Batt3/Meas1/'; --> 5
    './Batt3/Meas2/'; --> 6* ...
    './Batt4/Meas1/';
    './Batt5/Meas1/';
    './Batt6/Meas1/';
    './Batt7/Meas1/';
    './Batt8/Meas1/';
    './Batt9/Meas1/';
    './Batt10/Meas1/'; -->13
    };
```

# Data Selection

In [11]:

# Data acquition file to load from dateset folder
battery_list=[1,2,3,4,5,7,8,9,10,11,12] # Data acquitions 6,13 to be used for TEST]

# Model Training

## Single model Train/Validation split

### Single Model - NO data Augmentation

In [12]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model_NO_AUGMENTATION"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=1)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12]
model name: Paper_EIS_single_model_NO_AUGMENTATION
dataset row number: 110
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION
df_real shape: (110, 14)
df_img shape: (110, 14)
soc: 100
battery: 1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/Batt_1_100.png
soc: 090
battery: 1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/Batt_1_090.png
soc: 080
battery: 1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/Batt_1_080.png
soc: 070
battery: 1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/Batt_1_070.png
soc: 060
battery: 1
augmentation_index

Valley: 1.00e-03


epoch,train_loss,valid_loss,accuracy,time
0,3.876678,5.065520,0.121212,00:01


epoch,train_loss,valid_loss,accuracy,time
0,3.817390,3.690923,0.151515,00:01
1,3.537513,2.936899,0.151515,00:00
2,3.365734,2.735416,0.151515,00:00
3,3.372200,2.628784,0.181818,00:00
4,3.367665,2.598216,0.151515,00:00
5,3.390305,2.547084,0.151515,00:00
6,3.336335,2.533404,0.212121,00:00
7,3.269355,2.467403,0.242424,00:00
8,3.235266,2.389773,0.272727,00:00
9,3.139300,2.345685,0.303030,00:00


saved filename: Paper_EIS_single_model_NO_AUGMENTATION_1648967951.884598_SAVED.pth
Setting-up type transforms pipelines
Found 110 items
2 datasets of sizes 77,33
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/Batt_10_040.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/Batt_10_040.png
    applying RegexLabeller gives
      040
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(3)

Final sample: (PILImage mode=RGB size=432x288, TensorCategor

In [13]:
score_model(weights_filename,dl)

[[3 0 0 0 0 0 0 0 0 0]
 [0 4 1 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0]
 [0 1 0 2 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 2 0 0 1 0]
 [0 0 0 0 0 0 2 1 0 0]
 [0 0 0 0 0 2 2 1 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 2 0 1 0 0 0 0 0 2]]
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.8        0.2        0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.33333333 0.         0.66666667 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.5        0.         0.         0.         0.
  0.         0.         0.         0.5       ]
 [0.         0.         0.         0.         0.         0.66666667
  0.         0.         0.33333333 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.66666667 0.33333333 0.         0.        ]
 [0.        

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


learn.validate()


Model accuracy: 0.61
learn.get_preds()


TensorBase(0.6061)

### Single model - Data Augmentation Very low noise

In [14]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model_LOW_NOISE"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,NOISE_AMOUNT=1e-12,DATA_AUGMENTATION_FACTOR=10)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12]
model name: Paper_EIS_single_model_LOW_NOISE
dataset row number: 110
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE
df_real shape: (110, 14)
df_img shape: (110, 14)
soc: 100
battery: 1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/Batt_1_100.png
augmentation_index: 1
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/Batt_1101_100.png
augmentation_index: 2
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/Batt_1201_100.png
augmentation_index: 3
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/Batt_1301_100.png
augmentation_index: 4
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/Batt_1401_100.png
augmentation_index

Valley: 1.74e-03


epoch,train_loss,valid_loss,accuracy,time
0,3.102637,2.387795,0.206061,00:06


epoch,train_loss,valid_loss,accuracy,time
0,2.012944,1.653769,0.436364,00:06
1,1.878665,0.871921,0.672727,00:06
2,1.681323,0.696434,0.690909,00:06
3,1.466126,0.558987,0.809091,00:06
4,1.321263,0.594943,0.766667,00:06
5,1.153233,0.478316,0.827273,00:06
6,1.045410,0.469638,0.806061,00:06
7,0.910096,0.286359,0.927273,00:06
8,0.782917,0.187199,0.945455,00:06
9,0.731060,0.217278,0.927273,00:06


saved filename: Paper_EIS_single_model_LOW_NOISE_1648968346.767459_SAVED.pth
Setting-up type transforms pipelines
Found 1100 items
2 datasets of sizes 770,330
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/Batt_1103_070.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/Batt_1103_070.png
    applying RegexLabeller gives
      070
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(6)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(6))


Found 1100 item

In [15]:
score_model(weights_filename,dl)

[[28  0  0  0  0  0  0  0  0  0]
 [ 0 32  0  0  0  0  0  0  0  0]
 [ 0  0 35  0  0  0  0  0  0  0]
 [ 0  0  0 28  0  0  0  0  0  0]
 [ 0  0  0  0 33  0  0  0  0  0]
 [ 0  0  0  0  0 26  0  0  0  0]
 [ 0  0  0  0  0  0 30  0  0  0]
 [ 0  0  0  0  0  0  0 50  0  0]
 [ 0  0  0  0  0  0  0  0 31  0]
 [ 0  0  0  0  0  0  0  0  0 37]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
interpretation.most_confused()
interpretation.top_losses()
interpretation.print_classification_report()
              precision    recall  f1-score   support

         010       1.00      1.00      1.00        28
         020       1.00      1.00      1.00        32
         030       1.00      1.00      1.00        35
         040     

learn.validate()


Model accuracy: 1.00
learn.get_preds()


TensorBase(1.)

### Single Model - Data augmentation with realistic AGW noise amount in data acquisition simulation

In [16]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=10)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12]
model name: Paper_EIS_single_model
dataset row number: 110
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model
df_real shape: (110, 14)
df_img shape: (110, 14)
soc: 100
battery: 1
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/Batt_1_100.png
augmentation_index: 1
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/Batt_1101_100.png
augmentation_index: 2
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/Batt_1201_100.png
augmentation_index: 3
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/Batt_1301_100.png
augmentation_index: 4
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/Batt_1401_100.png
augmentation_index: 5
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/Batt_1501_100.png
augmentation_index: 6
/gdri

Valley: 1.74e-03


epoch,train_loss,valid_loss,accuracy,time
0,3.232908,2.455859,0.206061,00:06


epoch,train_loss,valid_loss,accuracy,time
0,2.403735,1.840188,0.348485,00:06
1,2.236044,1.384449,0.469697,00:06
2,2.107873,1.263283,0.530303,00:06
3,1.921447,1.230747,0.509091,00:06
4,1.785453,1.237161,0.512121,00:06
5,1.667359,1.131200,0.545455,00:06
6,1.558270,0.994375,0.624242,00:06
7,1.456721,1.343871,0.478788,00:06
8,1.394916,0.990793,0.615152,00:06
9,1.349360,1.197883,0.554545,00:06


saved filename: Paper_EIS_single_model_1648968761.170263_SAVED.pth
Setting-up type transforms pipelines
Found 1100 items
2 datasets of sizes 770,330
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/Batt_1103_070.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/Batt_1103_070.png
    applying RegexLabeller gives
      070
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(6)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(6))


Found 1100 items
2 datasets of sizes 770,330
Setting up Pipeline:

In [17]:
score_model(weights_filename,dl)

[[26  2  0  0  0  0  0  0  0  0]
 [ 0 24  2  1  2  0  0  0  0  3]
 [ 0  2 21  5  5  0  0  0  0  2]
 [ 0  1  5 20  1  0  0  0  0  1]
 [ 0  4  7  5 17  0  0  0  0  0]
 [ 0  0  0  0  0 24  1  0  1  0]
 [ 0  0  0  0  0  0 22  7  1  0]
 [ 0  0  0  0  0  0  9 36  5  0]
 [ 0  1  0  0  0  0  4  4 22  0]
 [ 0  3  0  1  0  0  0  0  0 33]]
[[0.92857143 0.07142857 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.75       0.0625     0.03125    0.0625     0.
  0.         0.         0.         0.09375   ]
 [0.         0.05714286 0.6        0.14285714 0.14285714 0.
  0.         0.         0.         0.05714286]
 [0.         0.03571429 0.17857143 0.71428571 0.03571429 0.
  0.         0.         0.         0.03571429]
 [0.         0.12121212 0.21212121 0.15151515 0.51515152 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.92307692
  0.03846154 0.         0.03846154 0.        ]
 [0.         0.    

learn.validate()


Model accuracy: 0.74
learn.get_preds()


TensorBase(0.7424)